In [1]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [2]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [3]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [4]:
# program the target
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 5879 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5879 bytes


In [5]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 2000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

Capturing traces: 100%|██████████| 2000/2000 [05:08<00:00,  6.55it/s]


In [6]:
from lascar import *
from lascar.tools.aes import sbox

def generate_selection_function(byte):
    def selection_with_guess(value, guess): # selection_with_guess function must take 2 arguments: value and guess
        #print("value: {}".format(value))
        return hamming(sbox[value[byte] ^ guess])
    return selection_with_guess

In [7]:
mycontainer =  TraceBatchContainer(trace_array, textin_array)

In [8]:
guess_range = range(256)

In [9]:
cpa_engines = [CpaEngine("cpa_%02d" % i, generate_selection_function(i), guess_range) for i in range(16)]

In [30]:
from lascar.output.parse_results import apply_parse
import pandas as pd
from IPython.display import clear_output
class JupyterOutputMethod(OutputMethod):
    def _update(self, engine, results):
        engine.output_parser_mode = "argmax"
        results_parsed = apply_parse(engine, abs(results))
        if results_parsed is None:
            return
        df = pd.DataFrame(results_parsed).sort_values(2, axis=0)
        fd = df.transpose()
        def formatter(stats):
            return str("{}".format(stats))
        clear_output(wait=True)
        display(fd.head().style.format(formatter))
        
session = Session(mycontainer, engines=cpa_engines, output_method=JupyterOutputMethod(*cpa_engines), output_steps=500).run(batch_size=100)

,60,63,15,190,201,234,29,245,27,239,33,0,188,155,236,9,113,49,200,204,202,42,231,112,129,131,246,115,2,55,160,92,132,19,219,146,118,3,73,72,54,138,53,232,128,163,212,218,107,31,7,119,182,127,209,181,11,67,59,150,136,158,222,91,65,117,135,51,79,214,244,6,254,208,76,168,25,223,149,89,21,116,220,71,17,221,242,81,77,240,24,40,43,47,175,125,174,46,180,44,108,172,152,185,210,140,12,252,203,194,211,105,123,206,191,74,109,178,226,177,165,250,205,57,192,39,5,37,173,198,14,216,183,95,82,35,130,151,157,170,156,45,227,96,80,184,142,167,179,228,85,215,169,247,88,230,217,237,87,99,162,148,251,32,196,241,111,255,122,134,70,61,248,186,243,225,238,195,8,69,18,68,101,75,103,50,207,126,166,104,1,34,224,86,145,144,41,147,176,62,4,83,197,199,164,143,189,121,66,64,229,16,233,93,171,141,110,38,13,97,78,249,235,124,106,36,20,90,94,100,253,84,187,56,98,193,159,22,114,120,48,30,137,102,10,154,213,52,139,23,26,153,133,28,58,161
0,60.0,63.0,15.0,190.0,201.0,234.0,29.0,245.0,27.0,239.0,33.0,0.0,188.0,155.0,236.0,9.0,113.0,49.0,200.0,204.0,202.0,42.0,231.0,112.0,129.0,131.0,246.0,115.0,2.0,55.0,160.0,92.0,132.0,19.0,219.0,146.0,118.0,3.0,73.0,72.0,54.0,138.0,53.0,232.0,128.0,163.0,212.0,218.0,107.0,31.0,7.0,119.0,182.0,127.0,209.0,181.0,11.0,67.0,59.0,150.0,136.0,158.0,222.0,91.0,65.0,117.0,135.0,51.0,79.0,214.0,244.0,6.0,254.0,208.0,76.0,168.0,25.0,223.0,149.0,89.0,21.0,116.0,220.0,71.0,17.0,221.0,242.0,81.0,77.0,240.0,24.0,40.0,43.0,47.0,175.0,125.0,174.0,46.0,180.0,44.0,108.0,172.0,152.0,185.0,210.0,140.0,12.0,252.0,203.0,194.0,211.0,105.0,123.0,206.0,191.0,74.0,109.0,178.0,226.0,177.0,165.0,250.0,205.0,57.0,192.0,39.0,5.0,37.0,173.0,198.0,14.0,216.0,183.0,95.0,82.0,35.0,130.0,151.0,157.0,170.0,156.0,45.0,227.0,96.0,80.0,184.0,142.0,167.0,179.0,228.0,85.0,215.0,169.0,247.0,88.0,230.0,217.0,237.0,87.0,99.0,162.0,148.0,251.0,32.0,196.0,241.0,111.0,255.0,122.0,134.0,70.0,61.0,248.0,186.0,243.0,225.0,238.0,195.0,8.0,69.0,18.0,68.0,101.0,75.0,103.0,50.0,207.0,126.0,166.0,104.0,1.0,34.0,224.0,86.0,145.0,144.0,41.0,147.0,176.0,62.0,4.0,83.0,197.0,199.0,164.0,143.0,189.0,121.0,66.0,64.0,229.0,16.0,233.0,93.0,171.0,141.0,110.0,38.0,13.0,97.0,78.0,249.0,235.0,124.0,106.0,36.0,20.0,90.0,94.0,100.0,253.0,84.0,187.0,56.0,98.0,193.0,159.0,22.0,114.0,120.0,48.0,30.0,137.0,102.0,10.0,154.0,213.0,52.0,139.0,23.0,26.0,153.0,133.0,28.0,58.0,161.0
1,0.8740076144056295,0.20427177596540322,0.20242019324342125,0.20046605152387653,0.18677429068818704,0.18628338449159715,0.183617353953263,0.18051036769867804,0.16946926777833943,0.16858054784878065,0.1668211493699261,0.16250052093623527,0.1612947399254598,0.1611436244777353,0.15938927747522402,0.158426406285605,0.1566155652964581,0.15582567943338257,0.15384395664433878,0.15147066369614368,0.15095005045166493,0.15081591301765807,0.14704848695626827,0.14685487478066364,0.1466707694609428,0.14592580085312215,0.14579940957024726,0.1457713924606242,0.14533207375644167,0.14442664516098427,0.14426533849701637,0.14406713169996443,0.14334557892528677,0.14303122825359266,0.14285247924120642,0.14271879051182146,0.14207784727273576,0.14198175930962553,0.14167970471940863,0.14160326225583866,0.14100298763847907,0.13997318795332034,0.13797615630685506,0.13723877954339556,0.13659920890893748,0.13640546694042763,0.1361404752221226,0.13466935575900849,0.13440997126476545,0.1341724378766639,0.13408964863875833,0.133929522617755,0.13365777179888913,0.13347534519501056,0.13333860480486018,0.13328104844318392,0.1330571647994144,0.13285254841736246,0.13132383110102439,0.12916982005974997,0.12896296650732667,0.12893502081649638,0.12862027056314643,0.12790787581581609,0.12751091855785612,0.12615962094251215,0.12611918331565874,0.12585405866612076,0.12576291620763505,0.1257365326458153,0.12450472078762474,0.12426002867047872,0.12387692222327114,0.12342041017652124,0.12268267493925962,0.12256268753085543,0.12238679143425872,0.12217139304672764,0.12202346250100146,0.1218959749282468,0.12155449249278305,0.12146801052271522,0.12135543395676866,0.12129518244

Session |100%||2000 trc/2000 | (18 engines, batch_size=100, leakage_shape=(5000,)) |Time:  0:00:20


AttributeError: 'JupyterOutputMethod' object has no attribute '_finalize'

In [69]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
for i in range(16):
    results = cpa_engines[i].finalize()
    xrange = range(len(results[0x2B]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess])
    
show(p)

Loading BokehJS ...

Best Guess is 2B (Corr = 0.8819549855894722)
Best Guess is 7E (Corr = 0.8641673643569957)
Best Guess is 15 (Corr = 0.8861143398259638)
Best Guess is 16 (Corr = 0.8881167317269975)
Best Guess is 28 (Corr = 0.8867853178287347)
Best Guess is AE (Corr = 0.8788261857956764)
Best Guess is D2 (Corr = 0.8726894924236483)
Best Guess is A6 (Corr = 0.8686279256542734)
Best Guess is AB (Corr = 0.8988567019274656)
Best Guess is F7 (Corr = 0.8841709148770263)
Best Guess is 15 (Corr = 0.8842457852361437)
Best Guess is 88 (Corr = 0.876149844333077)
Best Guess is 09 (Corr = 0.8935485562108787)
Best Guess is CF (Corr = 0.8658320176664109)
Best Guess is 4F (Corr = 0.8790855835734749)
Best Guess is 3C (Corr = 0.8747517031930832)


In [47]:
print(results)

[[-0.09475346 -0.05683607 -0.07875837 ... -0.09013332 -0.05793832
  -0.10748509]
 [-0.02612699  0.04846655  0.04347683 ...  0.185771    0.16331949
   0.15497122]
 [-0.01171707 -0.07218269 -0.00485762 ... -0.00583763  0.00855455
  -0.01280755]
 ...
 [-0.02969102  0.00199233 -0.02801654 ...  0.1284462   0.12930057
   0.12500131]
 [ 0.04364144  0.09195242  0.0383671  ...  0.00721142  0.01861461
   0.        ]
 [ 0.04094363 -0.06576713  0.03622526 ... -0.1391642  -0.14273133
  -0.10194352]]


In [68]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
xrange = range(len(results[0x2B]))
print(xrange)
print(len(results))
p = figure()
#p.line(xrange, traces[5],line_color='red')
p.line(xrange, results[0x3C])
show(p)

Loading BokehJS ...

range(0, 5000)
256


In [94]:
print(mycontainer.values)

[[219  98  78 ... 162 199  74]
 [228 136 198 ...  24 155  71]
 [111  52  64 ... 148  49 139]
 ...
 [244 166 212 ... 207 249 141]
 [ 77 175 127 ...  42 195  42]
 [101 146 191 ... 163 217  79]]
